# Load env

In [2]:
import os
os.chdir('..')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import importlib
from reports_etl import *
from enrich_holdings import *
from fossil_classification import *
from holdings_analysis import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [5]:
last_updated()

.py file last modified: 2021-12-16 18:36:31.498425


# Bulk download reports
Using the http response recorded while searching for reports here:
https://employersinfocmp.cma.gov.il/#/publicreports

In [20]:
# get reports from response.json search results
response_path = "data/downloaded reports/company reports 2021Q4/"
reports = get_reports_from_response(response_path)

Number of reports included in response.json: 59


## Check for missing reports

In [6]:
reports.head()

,EntityId,CodeFile,DocumentCode,DocumentName,DocumentId,fileExt,ReportFor,ReportPeriodCode,ReportPeriodDesc,EntHistLevel,SystemCode,FunctionCode,LicenseCode,LicenseStatusId,DateStart,DateFinish,ProductNum,LegalId,Name,ShortName,EventDate,StatusDate,SystemName,ReportHistLevel,EventId,DocStatus,EntityName,LicenseId,ParentCorpName,InvestmentNumber,ParentCorpLegalId,ParentCorpEntlId,IsMaxRowsExceeded,url
0,220,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2181007,xlsx,1,71000001,2021 רבעון 4,0,300003,0,0,0,nan,nan,0,570009449,nan,nan,01/10/2021,24/01/2022,גמל,0,624999,1,nan,0,"עוצ""מ - אגודה שיתופית לניהול קופות גמל בע""מ",nan,570009449,nan,False,https://employersinfocmp.cma.gov.il/api/Public...
1,219,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2184207,xlsx,1,71000001,2021 רבעון 4,0,300003,0,0,0,nan,nan,0,520034968,nan,nan,01/10/2021,30/01/2022,גמל,0,626177,1,nan,0,החברה המנהלת של קרן השתלמות של עובדי חברת החשמ...,nan,520034968,nan,False,https://employersinfocmp.cma.gov.il/api/Public...
2,5937,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2184890,xlsx,1,71000001,2021 רבעון 4,0,300003,0,0,0,nan,nan,0,514767490,nan,nan,01/10/2021,30/01/2022,גמל,0,626571,1,nan,0,"סלייס גמל בע""מ",nan,514767490,nan,False,https://employersinfocmp.cma.gov.il/api/Public...
3,209,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2185960,xlsx,1,71000001,2021 רבעון 4,0,300003,0,0,0,nan,nan,0,510694821,nan,nan,01/10/2021,31/01/2022,גמל,0,627222,1,nan,0,"לאומי קמ""פ בע""מ",nan,510694821,nan,False,https://employersinfocmp.cma.gov.il/api/Public...
4,293,73,71100073,רשימת נכסים ברמת נכס בודד- Public - חברה - גמל,2186050,xlsx,1,71000001,2021 רבעון 4,0,300003,0,0,0,nan,nan,0,520024985,nan,nan,01/10/2021,31/01/2022,גמל,0,627255,1,nan,0,"עגור חברה לניהול קופות גמל וקרנות השתלמות בע""מ",nan,520024985,nan,False,https://employersinfocmp.cma.gov.il/api/Public...


In [7]:
# filter specific reports for quick analysis
# reports = reports[reports["ParentCorpName"].str.startswith("מגדל")]

In [8]:
# download reports - run only once per response
download_reports(reports, response_path)

# Process multiple reports -> unified file

In [9]:
reports_fn_list = get_filename_list(response_path)

number of files to be pre-processed: 59


## Pre-process - check sheet names and column names, add to standardization if needed

In [10]:
pre_process_reports(reports_fn_list)

{'סכום נכסי הקרן': 59, 'מזומנים': 59, 'תעודות התחייבות ממשלתיות': 59, 'תעודות חוב מסחריות': 59, 'אג"ח קונצרני': 59, 'מניות': 59, 'קרנות סל': 59, 'קרנות נאמנות': 59, 'כתבי אופציה': 59, 'אופציות': 59, 'חוזים עתידיים': 59, 'מוצרים מובנים': 59, 'לא סחיר - תעודות התחייבות ממשלתי': 59, 'לא סחיר - תעודות חוב מסחריות': 59, 'לא סחיר - אג"ח קונצרני': 59, 'לא סחיר - מניות': 59, 'לא סחיר - קרנות השקעה': 59, 'לא סחיר - כתבי אופציה': 59, 'לא סחיר - אופציות': 59, 'לא סחיר - חוזים עתידיים': 59, 'לא סחיר - מוצרים מובנים': 59, 'הלוואות': 59, 'פקדונות מעל 3 חודשים': 59, 'זכויות מקרקעין': 59, 'השקעה בחברות מוחזקות': 59, 'השקעות אחרות': 59, 'יתרת התחייבות להשקעה': 59, 'עלות מתואמת אג"ח קונצרני סחיר': 59, 'עלות מתואמת אג"ח קונצרני ל.סחיר': 59, 'עלות מתואמת מסגרות אשראי ללווים': 59, '{PL}PickLst': 3}


,מזומנים,תעודות התחייבות ממשלתיות,תעודות חוב מסחריות,"אג""ח קונצרני",מניות,קרנות סל,קרנות נאמנות,כתבי אופציה,אופציות,חוזים עתידיים,מוצרים מובנים,לא סחיר - תעודות התחייבות ממשלתי,לא סחיר - תעודות חוב מסחריות,"לא סחיר - אג""ח קונצרני",לא סחיר - מניות,לא סחיר - קרנות השקעה,לא סחיר - כתבי אופציה,לא סחיר - אופציות,לא סחיר - חוזים עתידיים,לא סחיר - מוצרים מובנים,הלוואות,פקדונות מעל 3 חודשים,זכויות מקרקעין,השקעה בחברות מוחזקות,השקעות אחרות,"עלות מתואמת אג""ח קונצרני סחיר","עלות מתואמת אג""ח קונצרני ל.סחיר",עלות מתואמת מסגרות אשראי ללווים,{PL}PickLst
שם המנפיק/שם נייר ערך,59.00,59.00,59.00,59.00,59.00,59.00,58.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,57.00,59.00,59.00,59.00,59.00,59.00,nan
"מספר ני""ע",59.00,59.00,59.00,59.00,59.00,59.00,58.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,nan,nan,59.00,59.00,59.00,59.00,nan
מספר מנפיק,59.00,nan,59.00,59.00,59.00,59.00,58.00,nan,nan,nan,nan,nan,59.00,59.00,59.00,nan,nan,nan,nan,nan,59.00,59.00,nan,59.00,nan,nan,nan,nan,nan
דירוג,59.00,59.00,59.00,59.00,nan,nan,58.00,nan,nan,nan,59.00,59.00,59.00,59.00,nan,nan,nan,nan,nan,59.00,59.00,59.00,nan,59.00,59.00,59.00,59.00,59.00,nan
שם מדרג,59.00,59.00,59.00,59.00,nan,nan,58.00,nan,nan,nan,59.00,59.00,59.00,59.00,nan,nan,nan,nan,nan,59.00,59.00,59.00,nan,59.00,59.00,59.00,59.00,59.00,nan
סוג מטבע,59.00,59.00,59.00,59.00,59.00,59.00,58.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,57.00,59.00,59.00,59.00,59.00,59.00,nan
שעור ריבית,59.00,59.00,59.00,59.00,nan,nan,nan,nan,nan,nan,59.00,59.00,59.00,59.00,nan,nan,nan,nan,nan,59.00,59.00,59.00,nan,59.00,59.00,59.00,59.00,59.00,nan
תשואה לפדיון,59.00,59.00,59.00,59.00,nan,nan,nan,nan,nan,nan,59.00,59.00,59.00,59.00,nan,nan,nan,nan,nan,59.00,59.00,59.00,nan,59.00,59.00,nan,nan,nan,nan
שווי,59.00,59.00,59.00,59.00,59.00,59.00,58.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,57.00,59.00,59.00,59.00,59.00,59.00,nan
שעור מנכסי אפיק ההשקעה,59.00,59.00,59.00,59.00,59.00,59.00,58.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,59.00,57.00,59.00,59.00,59.00,59.00,59.00,nan


# process holding reports
1. summary sheets (for verification)
2. holding sheets - raw data

## 1. summary sheets

In [11]:
all_summary_sheets = process_summary_sheets(reports_fn_list)

In [12]:
totals = get_totals(all_summary_sheets)

Number of totals found: 59


In [13]:
totals["sum_num"].sum()

1776366424.8264937

## 2. Holdings

In [14]:
# extract all holdings from reports
all_holdings_clean = clean_holdings(extract_holdings(reports_fn_list))

Processing report 59 out of 59
before cleaning: 88752
 after cleaning: 80031


# add fund and company data to holdings

In [15]:
## Manually add missing corps to reports - only if needed, for funds processing
missing_corps = pd.DataFrame(columns=reports.columns)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'מנורה מבטחים ביטוח בע"מ',
#                                       'ParentCorpLegalId': '520042540',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)
# missing_corps = missing_corps.append({'SystemName': 'חיים ואובדן כושר עבודה',
#                                       'ParentCorpName': 'כלל חברה לביטוח בע"מ',
#                                       'ParentCorpLegalId': '520024647',
#                                       'ReportPeriodDesc': ''
#                                      }, ignore_index=True)

reports = reports.append(missing_corps)

In [16]:
all_holdings_final = add_report_data(all_holdings_clean, reports)

# Compare totals from summary sheets with totals from holdings

In [17]:
print("total from summary sheets: {:,.2f}".format(totals["sum_num"].sum()))
print("total from holdings sheets: {:,.2f}".format(all_holdings_final["שווי"].sum()))
print("diff: {:,.2f}".format(totals["sum_num"].sum() - all_holdings_final["שווי"].sum()))

total from summary sheets: 1,776,366,424.83
total from holdings sheets: 1,769,113,338.76
diff: 7,253,086.06


In [18]:
totals_from_holdings = pd.DataFrame(all_holdings_final.groupby("report_id")["שווי"].agg('sum')).reset_index()
totals_from_holdings.index = totals_from_holdings.index.astype('str')

In [19]:
totals_comparison = totals_from_holdings.merge(totals,
                                               on='report_id',
                                               how='outer'
                                              )

totals_comparison["diff"] = totals_comparison["sum_num"] - totals_comparison["שווי"]
totals_comparison[abs(totals_comparison["diff"]) > 1].sort_values("diff", ascending=False)

,report_id,שווי,asset,sum,pct,pct_num,sum_num,diff
45,2222108,"42,550,199.32","סה""כ סכום נכסי המסלול או הקרן","49,723,257.06",1,1,"49,723,257.06","7,173,057.74"
55,2225626,"213,128,906.45","סה""כ סכום נכסי המסלול או הקרן","213,189,352.44",1,1,"213,189,352.44","60,445.98"
11,2216901,"2,442,477.37","סה""כ סכום נכסי המסלול או הקרן","2,459,446.12",1,1,"2,459,446.12","16,968.75"
41,2221683,"80,583,303.57","סה""כ סכום נכסי המסלול או הקרן","80,584,342.30",1,1,"80,584,342.30","1,038.72"
58,2226019,"93,772,952.27","סה""כ סכום נכסי המסלול או הקרן","93,773,952.90",1,1,"93,773,952.90","1,000.63"
56,2225937,"48,704,983.60","סה""כ סכום נכסי המסלול או הקרן","48,705,555.83",1,1,"48,705,555.83",572.23


In [20]:
# report drill down
def report_sums_by_type(report_id):
    rep = all_holdings_final[all_holdings_final["report_id"] == report_id]
    print("total sum by holdings: {:,.2f}".format(rep["שווי"].sum()))
    return pd.DataFrame(rep.groupby("holding_type")["שווי"].agg('sum').map('{:,.2f}'.format))
    
report_sums_by_type('2222108')

total sum by holdings: 42,550,199.32


,שווי
holding_type,
"אג""ח קונצרני","2,557,722.98"
אופציות,"95,780.07"
הלוואות,"1,996,889.98"
השקעות אחרות,"-1,353.65"
זכויות מקרקעין,"246,315.01"
חוזים עתידיים,"27,735.64"
כתבי אופציה,"46,987.83"
"לא סחיר - אג""ח קונצרני","313,755.14"
לא סחיר - אופציות,"27,259.27"


### totals discrepancies
<u>2021Q2</u><br>
* In report 2128339 - there's a redundant line in נדלן, consider removing manually when all files for Q3 are here - removing
* In report 2153994 - the holdings in מזומנים are mostly missing. adding an artificial row. (הפניקס)
* In report 2146293 - the value of holdings in לא סחיר - אג"ח קונצרני are way off. leaving as is for now.

<u>2021Q3</u>
* 2148642 - missing exactly 10000 in מזומנים. leaving as is.
* 2148994 - dropped a line in הלוואות withno security number. not fixing for now.

<u>2021Q3</u>
* 2222108 - the holdings in מזומנים are mostly missing. adding an artificial row. (הפניקס)


## Removing redundant rows

In [20]:
remove_rows = (
    (all_holdings_final["report_id"] == '2128339') &
    (all_holdings_final["holding_type"] == 'זכויות מקרקעין') &
    (all_holdings_final["שעור מנכסי אפיק ההשקעה"] == 1)
)

all_holdings_final = all_holdings_final.drop(all_holdings_final[remove_rows].index, axis=0)

# all_holdings_final.groupby("report_id").filter(
#     lambda x: x["שעור מנכסי אפיק ההשקעה"].sum() > 1
# ).head()

## Adding missing rows

In [20]:
# adding a row for report 2153994, to compensate for missing holding values
total_from_summary = all_summary_sheets[
    (all_summary_sheets["report_id"] == '2222108') &
    (all_summary_sheets["asset"]).str.contains('מזומנים')
]["sum_num"]

holdings_to_fix = all_holdings_final[
    (all_holdings_final["report_id"] == '2222108') &
    (all_holdings_final["holding_type"] == 'מזומנים')
]

total_from_holdings = holdings_to_fix["שווי"].sum()

# calculate sum and pct to be added
to_be_added_sum = float(total_from_summary) - float(total_from_holdings)
to_be_added_pct = 1 - holdings_to_fix["שעור מנכסי אפיק ההשקעה"].sum()
to_be_added_pct_of_report = float(to_be_added_sum) / float(totals[totals["report_id"] == '2222108']["sum_num"])

# prepare new row - take the highest amount row as base and edit it
new_row = holdings_to_fix.sort_values("שווי", ascending=False).head(1)
new_row["שם המנפיק/שם נייר ערך"] = "unknown - added to match sum"
new_row['שווי'] = to_be_added_sum
new_row['שעור מנכסי אפיק ההשקעה'] = to_be_added_pct
new_row['שעור מסך נכסי השקעה'] = to_be_added_pct_of_report

for col in ['מספר ני"ע', 'מספר מנפיק', 'דירוג', 'שם מדרג', 'סוג מטבע']:
    new_row[col] = None

print("adding row: ")
new_row

adding row: 


,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc
63490,unknown - added to match sum,None,None,None,None,None,0,0,"7,172,886.70",0.97,0.14,2222108,מזומנים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"הפניקס פנסיה וגמל בע""מ",513026484,0,NaN,NaN,31/03/2022,2021 רבעון 4


In [21]:
# adding the artificial row
all_holdings_final = all_holdings_final.append(new_row)

In [22]:
all_holdings_final.to_csv(response_path+"all_holdings.csv", index=False)

In [23]:
all_holdings_ever = all_holdings_final

## Export file for classification

In [24]:
cols_for_cls = [
    'שם המנפיק/שם נייר ערך', 'מספר ני"ע', 'מספר מנפיק',
    'שווי' ,'שעור מנכסי אפיק ההשקעה', 'שעור מסך נכסי השקעה', 
       'holding_type', 'זירת מסחר', 'תאריך רכישה', 'ערך נקוב', 'שער','שעור מערך נקוב מונפק',
       'ענף מסחר',
       'SystemName', 'ParentCorpName', 'ReportPeriodDesc'
]

holdings_for_cls = all_holdings_final[
    all_holdings_final["holding_type"].isin(['מניות', 'אג"ח קונצרני'])
]

holdings_for_cls[cols_for_cls].to_csv(response_path+"holdings_for_cls.csv", index=False)

In [29]:
holdings_for_cls[
    (holdings_for_cls['מספר ני"ע'].astype(str) == '11616780')
]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc
56194,אלדן תחבורה אגח ו-הפחתה,11616780,510454333,ilA,S&P מעלות,שקל חדש,0.03,0.00,324.90,0.00,0,2222108,"אג""ח קונצרני",TASE,30/04/21,2.88,"324,058.60",100.26,0,0.00,NaN,שירותים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"הפניקס פנסיה וגמל בע""מ",513026484,0,NaN,NaN,31/03/2022,2021 רבעון 4
65367,אלדן תחבורה אגח ו-הפחתה,11616780,510454333,ilA,S&P מעלות,שקל חדש,0.03,0.00,"3,562.14",0.00,0,2221683,"אג""ח קונצרני",TASE,30/04/21,2.88,"3,552,903.03",100.26,0,0.01,NaN,שירותים,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ביטוח,"הפניקס חברה לביטוח בע""מ",520023185,0,NaN,NaN,31/03/2022,2021 רבעון 4


## add new processed holdings to all_holdings file

In [10]:
# reload results from file
all_holdings_path = fetch_all_holdings_path()
new_holdings_path = "data/downloaded reports/company reports 2021Q4/all_holdings.csv"
all_holdings_ever = concat_from_csv_by_path(all_holdings_path, new_holdings_path)

pd.DataFrame(all_holdings_ever.groupby(['ReportPeriodDesc'])['שווי'].agg('sum').map('{:,.2f}'.format))

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,שווי
ReportPeriodDesc,
2020 רבעון 1,"1,222,418,614.88"
2020 רבעון 2,"1,307,193,119.07"
2020 רבעון 3,"1,373,544,876.41"
2020 רבעון 4,"1,461,378,236.82"
2021 רבעון 1,"1,548,168,300.50"
2021 רבעון 2,"1,593,940,074.16"
2021 רבעון 3,"1,640,889,505.45"
2021 רבעון 4,"1,776,286,225.46"


In [11]:
# update all holdings file - handle with care
# TODO: add backup process
all_holdings_ever.to_csv(fetch_all_holdings_path(), index=False)

# Validate holdings file

In [34]:
all_holdings_curr_q = all_holdings_ever[all_holdings_ever["ReportPeriodDesc"] == '2021 רבעון 4']
# pd.DataFrame(all_holdings_curr_q.groupby(['SystemName', 'ParentCorpName', 'report_id'])['שווי'].agg('sum').map('{:,.2f}'.format))
pd.DataFrame(all_holdings_curr_q.groupby(['ParentCorpName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
ParentCorpName,
"איי. די. איי. חברה לביטוח בע""מ","320,449.63"
ארם גמולים - חברה לניהול קופות גמל בע''מ,"1,237,284.76"
"גל -ניהול קופות גמל לעובדי הוראה בע""מ","4,768,951.39"
"החברה המנהלת של מינהל קרן ההשתלמות לפקידים עובדי המנהל והשירותים בע""מ","931,420.12"
"החברה המנהלת של קרן השתלמות של עובדי חברת החשמל לישראל בע""מ","727,422.89"
"החברה המנהלת של רום קרן ההשתלמות לעובדי הרשויות המקומיות בע""מ","4,274,459.64"
"החברה לניהול קופת התגמולים והפנסיה של עובדי הסוכנות היהודית לארץ ישראל בע""מ","5,294,090.65"
"החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע""מ","2,144,250.78"
"החברה לניהול קרן השתלמות לביוכימאים ומקרוביולוגים בע""מ","314,045.61"


## Look for missing company reports

In [25]:
reports_cnt = pd.DataFrame(all_holdings_ever.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId'])['ReportPeriodDesc'].nunique().reset_index())
period_cnt = all_holdings_ever["ReportPeriodDesc"].nunique()

companies_with_missing_reports = reports_cnt[reports_cnt["ReportPeriodDesc"] < period_cnt]

In [26]:
available_reports_periods = pd.DataFrame(all_holdings_ever.groupby(['SystemName', 'ParentCorpName', 'ParentCorpLegalId', 'ReportPeriodDesc'])['שווי'].sum()).reset_index()
companies_w_missing_reports_available_reports = pd.merge(
    companies_with_missing_reports[['ParentCorpLegalId', 'SystemName']],
    available_reports_periods,
    on = ['ParentCorpLegalId', 'SystemName'],
    how="inner"
)
companies_w_missing_reports_available_reports

,SystemName,ParentCorpName,ParentCorpLegalId,ReportPeriodDesc,שווי


Located missing reports for major companies, manually downloaded

# Add fossil classification
Based on quarterly company level reports classification

## get previous fossil classifications

In [12]:
prev_cls_fn = "data_sources/prev_class.csv"
prev_by_sec_num, prev_by_ISIN = get_latest_fossil_classifications(prev_cls_fn)

previously classified Israeli security nums by is_fossil:
0.00    2306
1.00     232
nan        1
Name: is_fossil, dtype: int64
previously classified ISINs by is_fossil:
0.00    6653
1.00     555
Name: is_fossil, dtype: int64


## directly using previous company reports
Could be useful for fund level reports, commented out for now

In [13]:
# # TODO: using different file per holding_type for now, should change
# q_company_report1 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות באגח קונצרני.csv')
# q_company_report2 = pd.read_csv('/Users/urimarom/Downloads/חשיפה לפוסיליים - Q2 2021 - החזקות ישירות במניות.csv')

# q_company_report = pd.concat([
#     q_company_report1[['מספר ני"ע', 'is_fossil']],
#     q_company_report2[['מספר ני"ע', 'is_fossil']]
# ])

# q_company_report = q_company_report.drop_duplicates(subset=['מספר ני"ע']).set_index('מספר ני"ע')
# q_company_report.index = q_company_report.index.astype('str')

## Using previous classification

In [22]:
all_holdings_ever = pd.read_csv(fetch_all_holdings_path(), dtype=str)

In [24]:
all_holdings_cls = add_fossil_classifications(all_holdings_ever, prev_by_sec_num, prev_by_ISIN)

all_holdings: 666242
having holding number: 563510
without holding number: 102732


/Users/urimarom/PycharmProjects/fossil_classification/reports_etl.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holdings_with_num['מספר ני"ע'] = holdings_with_num['מספר ני"ע'].astype('str').str.strip().str.upper()


Holdings after fossil classification by Israeli security num:
nan     387167
0.00    154283
1.00     22060
Name: is_fossil, dtype: int64
Holdings after fossil classification by ISIN:
nan     304398
0.00    231154
1.00     27958
Name: is_fossil, dtype: int64
total fossil sum: 424941867.95378697
holdings count before classification: 666242
holdings count after classification: 666242


## Validate Fossil Classification results

In [29]:
# look for fossil holdings that aren't מניות or אג"ח קונצרני
all_holdings_cls[
    (all_holdings_cls["is_fossil"] > 0) &
    (~all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני']))
]

# looking good!
# TODO: might want to try and catch additional fossil holdings of other types - join by all id types

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
915,רותם אנרגיה אפ 3,1155746,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.20,0.00,0.00,2224705,כתבי אופציה,TASE,NaN,NaN,2550,8,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ביטוח,"איילון חברה לביטוח בע""מ",520042169,0,NaN,NaN,05/04/2022,2021 רבעון 4,1.00,0.20
1349,שיכון ובינוי6 מתואם,1129733,NaN,ilA,S&P מעלות,שקל חדש,0.04,NaN,514.02,0.07,0.00,2224705,"עלות מתואמת אג""ח קונצרני סחיר",NaN,09/09/2013,1.92,"482,115.93",NaN,NaN,0.00,NaN,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,0.03,ביטוח,"איילון חברה לביטוח בע""מ",520042169,0,NaN,NaN,05/04/2022,2021 רבעון 4,1.00,514.02
3098,SPDR S&P OIL &,US78468R5569,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,"7,385.08",0.00,0.00,2222107,קרנות סל,NYSE,NaN,NaN,"24,769.22",9587,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הפניקס פנסיה וגמל בע""מ",513026484,0,NaN,NaN,31/03/2022,2021 רבעון 4,1.00,"7,385.08"
4066,ORGS US - אופ לא סחירה 20.1.23,707705711,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,82.80,0.01,0,2222107,לא סחיר - אופציות,NaN,31/05/2021,NaN,"110,932.55",24,NaN,0.00,NaN,ל.ר.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"הפניקס פנסיה וגמל בע""מ",513026484,0,NaN,NaN,31/03/2022,2021 רבעון 4,1.00,82.80
10559,רותם אנרגיה אפ3,1155746,NaN,NaN,NaN,שקל חדש,NaN,NaN,0.11,0.00,0,2220462,כתבי אופציה,TASE,NaN,NaN,1400,8,NaN,0.00,NaN,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"רעות חברה לניהול קופות גמל בע""מ",510806870,0,NaN,NaN,30/03/2022,2021 רבעון 4,1.00,0.11
11494,XOP US SPDR S&P Oil & Gas Expl- SPDR S&P OIL &...,US78468R5569,,NaN,NaN,דולר אמריקאי,NaN,NaN,175.77,0.00,0.00,2217005,קרנות סל,NYSE,NaN,NaN,587.57,9619,0,0,NaN,Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"מנורה מבטחים פנסיה וגמל בע""מ",512245812,0,NaN,NaN,24/03/2022,2021 רבעון 4,1.00,175.77
15747,אלומיי אפ 1,1169325,NaN,NaN,NaN,שקל חדש,NaN,NaN,22.99,0.00,0,2195188,כתבי אופציה,TASE,NaN,NaN,4015,572.60,NaN,0.01,NaN,אנרגיה מתחדשת,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ביטוח,"הכשרה חברה לביטוח בע""מ",520042177,0,NaN,NaN,15/02/2022,2021 רבעון 4,1.00,22.99
19672,"שפיר הנדסה אג""ח ב'",1141951,NaN,ilA+,S&P מעלות,שקל חדש,0.03,0,0.00,0.00,0.00,2222993,השקעות אחרות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"הראל פנסיה וגמל בע""מ",512267592,0,NaN,NaN,03/04/2022,2021 רבעון 4,1.00,0.00
19681,דור אלון ה',1136761,NaN,A3.il,מידרוג,שקל חדש,0.05,0,34.81,0.00,0.00,2222993,השקעות אחרות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,פנסיה,"הראל פנסיה וגמל בע""מ",512267592,0,NaN,NaN,03/04/2022,2021 רבעון 4,1.00,34.81
20787,XOP US SPDR S&P Oil & Gas Expl- SPDR S&P OIL &...,US78468R5569,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,"70,573.45",0.02,0.00,2219991,קרנות סל,NYSE,NaN,NaN,"236,700.00","9,587.00",0,0.01,NaN,Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ביטוח,"מנורה מבטחים ביטוח בע""מ",520042540,0,NaN,NaN,30/03/2022,2021 רבעון 4,1.00,"70,573.45"


In [18]:
should_be_cls = all_holdings_cls[all_holdings_cls["holding_type"].isin(["מניות", 'אג"ח קונצרני'])]
missing_fossil_cls = pd.DataFrame(
    should_be_cls["is_fossil"].isnull().groupby([should_be_cls["ParentCorpName"], should_be_cls["ReportPeriodDesc"]]).sum()
)
missing_fossil_cls

is_fossil
ParentCorpName                                     ReportPeriodDesc           
איי. די. איי. חברה לביטוח בע"מ                     2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
איילון חברה לביטוח בע"מ                            2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
אינפיניטי ניהול השתלמות וגמל בע"מ                  2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
אלטשולר שחם גמל ופנסיה בע"מ                        2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
אנליסט קופות גמל בע"מ                              2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
ארם גמולים - חברה לניהול קופות גמל בע''מ           2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                                   2020 רבעון 3              0
                                                   2020 רבעון 4              0
                                                   2021 רבעון 1              0
                                                   2021 רבעון 2              0
                                                   2021 רבעון 3              0
                                                   2021 רבעון 4              0
גל -ניהול קופות גמל לעובדי הוראה בע"מ              2020 רבעון 1              0
                                                   2020 רבעון 2              0
                                        

## Write classified results to CSV & create another file with added id types

In [ ]:
all_holdings_cls.to_csv(response_path+"all_holdings_cls.csv", index=False)
# update all_holdings_cls_all_ids if needed
all_holdings_cls_all_ids = load_mappings_and_add_ids_to_holdings(all_holdings_cls)
all_holdings_cls_all_ids.to_csv(response_path+"all_holdings_cls_all_ids.csv", index=False)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


## Get Fund stats

In [50]:
fund_stats = get_summary(all_holdings_cls, 'ParentCorpName', 'SystemName', 'ProductNum','ShortName')
fund_stats.sort_values('שווי פוסילי', ascending=False).head()

,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
831,"מנורה מבטחים פנסיה וגמל בע""מ",פנסיה,2009,מנורה מבטחים פנסיה כללי,"97,517,828.84","3,688,669.73",0.04,"29,373,735.39","3,688,669.73",0.13,"36,290,878.50",0.06
531,"מגדל חברה לביטוח בע""מ",ביטוח,17012,NaN,"75,632,465.16","2,936,796.68",0.04,"27,579,070.32","2,936,796.68",0.11,"11,209,943.92",0.08
585,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",פנסיה,2102,מקפת אישית כללי,"63,232,444.53","1,807,664.74",0.03,"18,360,816.57","1,807,664.74",0.10,"22,850,094.41",0.04
447,"כלל חברה לביטוח בע""מ",ביטוח,14012,NaN,"44,269,431.59","1,725,016.47",0.04,"17,900,611.43","1,725,016.47",0.10,"8,481,832.16",0.07
503,"כלל פנסיה וגמל בע""מ",פנסיה,2002,מיטבית עתודות - ספיר,"53,073,440.42","1,608,629.20",0.03,"17,753,944.08","1,608,629.20",0.09,"17,847,738.78",0.05


In [51]:
fund_stats.to_csv(response_path+"fund_stats.csv", index=False)

In [373]:
# get fossil holdings per fund

fund_stats[fund_stats["ProductNum"].duplicated()]
# TODO: there is 1 duplicate, fund number 103 - check it later
def get_fossil_holdings(fund_id):
    fossil_holdings = all_holdings_cls[
        (all_holdings_cls["ProductNum"] == fund_id) &
        (all_holdings_cls["is_fossil"] > 0)
    ]
    return fossil_holdings.sort_values("שווי פוסילי", ascending=False)

get_fossil_holdings(11366)

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי
528808,איי.סי.אל,281014,520027830,NaN,NaN,שקל חדש,NaN,NaN,931.63,0.03,0.01,2082486,מניות,TASE,NaN,NaN,42136,2211,0,0,אחר,"כימיה, גומי ופלסטיק",NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,931.63
528800,שיכון ובינוי,1081942,520036104,NaN,NaN,שקל חדש,NaN,NaN,526.39,0.02,0.01,2082486,מניות,TASE,NaN,NaN,24971,2108,0,0.00,אחר,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,526.39
528827,פז נפט,1100007,510216054,NaN,NaN,שקל חדש,NaN,NaN,494.25,0.02,0.01,2082486,מניות,TASE,NaN,NaN,1235,40020,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,494.25
528839,אלקו החזקות,694034,520025370,NaN,NaN,שקל חדש,NaN,NaN,467.15,0.02,0.00,2082486,מניות,TASE,NaN,NaN,2408,19400,0,0.00,אחר,השקעות ואחזקות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,467.15
528881,דור אלון,1093202,520043878,NaN,NaN,שקל חדש,NaN,NaN,398.29,0.01,0.00,2082486,מניות,TASE,NaN,NaN,4291,9282,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,398.29
528661,שיכון ובינוי אגח 8,1135888,520036104,ilA,S&P מעלות,שקל חדש,0.04,0.01,354.72,0.02,0.00,2082486,"אג""ח קונצרני",TASE,22/01/18,5.36,"290,278.67",122.20,0,0.00,אחר,בנייה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,354.72
528608,חשמל אגח 27,6000210,520000472,Aa1.il,מידרוג,שקל חדש,0.04,-0.00,335.17,0.02,0.00,2082486,"אג""ח קונצרני",TASE,08/03/17,6.09,"253,170.36",132.39,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,335.17
528749,רציו מימון אגח ב,1139443,515060044,0,לא מדורג,שקל חדש,0.02,0.03,318.40,0.02,0.00,2082486,"אג""ח קונצרני",TASE,30/01/17,1.06,240614,132.33,0,0.00,אחר,חיפושי נפט וגז,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,318.40
528826,בזן,2590248,520036658,NaN,NaN,שקל חדש,NaN,NaN,313.00,0.01,0.00,2082486,מניות,TASE,NaN,NaN,372619,84,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,26/07/2021,2021 רבעון 2,1.00,313.00
528607,חשמל אגח 29,6000236,520000472,Aa1.il,מידרוג,שקל חדש,0.04,-0.01,307.10,0.02,0.00,2082486,"אג""ח קונצרני",TASE,24/06/19,3.84,240000,127.96,0,0.00,אחר,אנרגיה,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"אנליסט קופות גמל בע""מ",511880460,11366,אנליסט מסלולית - קופת גמל להשקעה לחיסכון ארוך ...,אנליסט חיסכון לילד - חוס

## חסכון לכל ילד

In [116]:
fund_stats[
    fund_stats["ShortName"].str.contains("ילד", na=False)
].sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים", ascending=False).reset_index()

,index,ParentCorpName,SystemName,ProductNum,ShortName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
0,617,"מור קופות גמל בע""מ",גמל,9303,מור חיסכון לילד - שריעה,"12,057.49",642.07,0.05,"1,346.55",642.07,0.48,"10,234.77",0.06
1,87,"אנליסט קופות גמל בע""מ",גמל,11366,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון בינוני,"101,480.54","8,877.56",0.09,"46,278.02","8,877.56",0.19,"28,634.65",0.12
2,799,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11321,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"345,064.19","15,913.25",0.05,"92,284.62","15,913.25",0.17,"167,272.38",0.06
3,86,"אנליסט קופות גמל בע""מ",גמל,11365,אנליסט חיסכון לילד - חוסכים המעדיפים סיכון מועט,"300,371.10","15,566.67",0.05,"93,063.87","15,566.67",0.17,"153,291.86",0.06
4,255,"הפניקס פנסיה וגמל בע""מ",גמל,11310,אקסלנס חיסכון לילד - חוסכים המעדיפים סיכון מועט,"305,991.72","20,260.19",0.07,"141,330.13","20,260.19",0.14,"123,964.03",0.08
5,800,"מנורה מבטחים פנסיה וגמל בע""מ",גמל,11322,מנורה מבטחים חסכון לילד - חוסכים המעדיפים סיכו...,"101,138.09","5,622.95",0.06,"39,773.41","5,622.95",0.14,"27,824.37",0.08
6,927,"פסגות קופות גמל ופנסיה בע""מ",גמל,11380,פסגות חיסכון לילד - חוסכים המעדיפים סיכון מועט,"347,506.79","8,409.53",0.02,"60,142.39","8,409.53",0.14,"233,995.80",0.03
7,182,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11354,הלמן-אלדובי חיסכון לילד- חוסכים המעדיפים סיכון...,"295,838.76","23,549.21",0.08,"169,382.58","23,518.02",0.14,"94,283.86",0.09
8,183,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",גמל,11355,הלמן-אלדובי חיסכון לילד- חוסכים המעדיפים סיכון...,"48,501.64","3,326.00",0.07,"25,339.70","3,308.10",0.13,"16,996.30",0.08
9,568,"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ",גמל,9896,מגדל חסכון לילד - חוסכים המעדיפים סיכון מועט,"337,849.08","17,474.94",0.05,"136,904.78","17,474.94",0.13,"120,102.40",0.07


In [411]:
all_holdings_cls.groupby(["SystemName","ParentCorpName","ProductNum"])["שעור מסך נכסי השקעה"].sum()

SystemName              ParentCorpName                         ProductNum
גמל                     אלטשולר שחם גמל ופנסיה בע"מ            472          1.00
                                                               558          1.00
                                                               588          1.00
                                                               1093         1.00
                                                               1094         1.00
                                                               1290         1.00
                                                               1375         1.00
                                                               1376         1.00
                                                               1377         1.00
                                                               1378         1.00
                                                               1394         1.00
                                   

## Fund level data - Discrepancies
* הלמן - seems like we looked at the wrong company level report - it was חברת חשמל...
<br>The numbers ~fit the ones in the company level PDF report. The gov site is missing a company level report.

the company name is: הלמן-אלדובי קופות גמל ופנסיה בע"מ


In [78]:
# missing reports
[r for r in funds_df.index.unique() if r not in all_holdings_final["report_id"].unique()]

# checked 2 missing, both have 0 holdings, all good

NameError: name 'funds_df' is not defined

In [28]:
pd.DataFrame(all_holdings_final.groupby(['ParentCorpName', 'SystemName'])["report_id"].nunique())

report_id
ParentCorpName                                     SystemName                       
איי. די. איי. חברה לביטוח בע"מ                     חיים ואובדן כושר עבודה          3
איילון חברה לביטוח בע"מ                            חיים ואובדן כושר עבודה         12
אלטשולר שחם גמל ופנסיה בע"מ                        גמל                            30
                                                   פנסיה                          14
אנליסט קופות גמל בע"מ                              גמל                            31
ארם גמולים - חברה לניהול קופות גמל בע''מ           גמל                             3
גל -ניהול קופות גמל לעובדי הוראה בע"מ              גמל                             9
החברה המנהלת של מינהל קרן ההשתלמות לפקידים עובד... גמל                             3
החברה המנהלת של קרן השתלמות של עובדי חברת החשמל... גמל                             1
החברה המנהלת של רום קרן ההשתלמות לעובדי הרשויות... גמל                             4
החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע"מ    גמל                             3
החברה לניהול קרן ההשתלמות לעובדי המדינה בע"מ       גמל                             5
החברה לניהול קרן השתלמות לאקדמאים במדעי החברה ו... גמל                             3
החברה לניהול קרן השתלמות לביוכימאים  ומקרוביולו... גמל                             2
החברה לניהול קרן השתלמות למשפטנים בע"מ             גמל                             3
החברה לניהול קרן השתלמות לשופטים בע"מ              גמל                             1
הכשרה חברה לביטוח בע"מ                             חיים ואובדן כושר עבודה         21
הלמן - אלדובי קופות גמל ופנסיה בע"מ                גמל                            39
הנדסאים וטכנאים - חברה לניהול קופות גמל בע"מ       גמל                             3
הפניקס חברה לביטוח בע"מ                            ביטוח                           2
                                                   חיים ואובדן כושר עבודה         28
הפניקס פנסיה וגמל בע"מ                             גמל                            46
                                                   פנסיה                          25
הראל חברה לביטוח בע"מ                              ביטוח                           4
                                                   חיים ואובדן כושר עבודה         30
הראל פנסיה וגמל בע"מ                               גמל                            43
                                                   פנסיה                          43
חברה לניהול קופות גמל של העובדים בעיריית תל - א... גמל                             3
חברת הגמל לעובדי האוניברסיטה העברית בע"מ           גמל                             5
יהב - פ.ר.ח. - חברה לניהול קופות גמל בע"מ          גמל                             2
יהב אחים ואחיות - חברה לניהול קופות גמל בע"מ       גמל                             2
יהב רופאים - חברה לניהול קופות גמל בע"מ            גמל                             3
ילין לפידות ניהול קופות גמל בע"מ                   גמל                            20
כלל חברה לביטוח בע"מ                               ביטוח                           2
                                                   חיים ואובדן כושר עבודה         19
כלל פנסיה וגמל בע"מ                                גמל                            36
                                                   פנסיה                          24
לאומי קמ"פ בע"מ                                    גמל                             1
מבטחים מוסד לביטוח סוציאלי של העובדים בע"מ         גמל                             1
מגדל חברה לביטוח בע"מ                              ביטוח                           4
                                                   חיים ואובדן כושר עבודה         15
מגדל מקפת קרנות פנסיה וקופות גמל בע"מ              גמל                            38
                                                   פנסיה                          27
מור קופות גמל בע"מ                                 גמל                            36
מחוג - מינהל גמל לעובדי חברת חשמל לישראל בע"מ      גמל                             4
מחר - חברה לניהול קופות גמל בע"מ                   גמל                          

### discrepancies

* report 2101262: empty sheet "זכויות מקרקעין" - explains all the diff
* report 2101276: all holding_types sums are ok, the sum @summary sheet is slightly different for some reason

In [123]:
print(funds_df[funds_df["ProductNum"] == 2207])
totals[totals["report_id"] == '2101262']

        SystemName                         ParentCorpName ParentCorpLegalId  \
2101262      פנסיה  מגדל מקפת קרנות פנסיה וקופות גמל בע"מ         512237744   

        ProductNum                                 Name  \
2101262       2207  מגדל מקפת אישית  למקבלי קצבה קיימים   

                             ShortName  StatusDate ReportPeriodDesc  
2101262  מקפת אישית למקבלי קצבה קיימים  29/08/2021     2021 רבעון 2  


,asset,sum,pct,report_id,pct_num,sum_num
4065,"סה""כ סכום נכסי המסלול או הקרן",3.58272e+06,1,2101262,1.0,3.582724e+06


# looking for lost insurance funds

### Migdal

In [37]:
migdal_ins = all_holdings_final[(all_holdings_final["ParentCorpLegalId"] == '520004896')]
migdal_by_fund = pd.DataFrame(migdal_ins.groupby(["ShortName", "ProductNum", "report_id"], dropna=False)["שווי"].agg('sum'))
migdal_by_fund.reset_index()

,ShortName,ProductNum,report_id,שווי
0,מגדל - כללי,17013,2149427,"35,047,120.77"
1,"מגדל אג""ח",8659,2149420,"93,094.92"
2,"מגדל אג""ח משולב מניות",74,2149430,"1,982,709.57"
3,מגדל בסיסי למקבלי קצבה,9606,2149424,"5,828,863.43"
4,מגדל מחקה מדד S&P 500,13625,2149423,"74,927.03"
5,"מגדל מסלול אג""ח ממשלת ישראל",68,2149417,"678,881.44"
6,"מגדל מסלול אג""ח עד 10% מניות",69,2149419,"3,208,931.08"
7,מגדל מסלול הלכה,185,2149418,"649,674.93"
8,מגדל מסלול לבני 50 ומטה,9599,2149422,"4,303,822.99"
9,מגדל מסלול לבני 50 עד 60,9604,2149433,"2,059,060.55"


https://www.migdal.co.il/about/investments/assets - there are 4 more funds not included in the reports site:
* מגדל משתתף ברווחים - קרן ח - b010_p221_520004896
* מגדל משתתף ברווחים - קרן ט - 520004896_b011_p221
* מגדל משתתף ברווחים - קרן י - b012_p221_520004896
* מגדל משתתף קרן י החדשה - b013_p221_520004896

Manually downloaded, added to the directory.

In [ ]:
pd.DataFrame(all_holdings_final[
    (all_holdings_final["ParentCorpLegalId"].isnull()) |
    (all_holdings_final["ParentCorpLegalId"] == 520004896)
].groupby("report_id")['שווי'].agg('sum').map('{:,.2f}'.format))

### Menora missing reports

In [118]:
pd.DataFrame(all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ'
].groupby(["ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)).sort_values("ProductNum").reset_index()

,ShortName,ProductNum,שווי
0,מנורה ביטוח שיקלי טווח קצר,41,"110,717.59"
1,"מנורה ביטוח אג""ח צמוד מדד",42,"1,210,425.63"
2,"מנורה ביטוח מט""ח",43,"29,976.13"
3,מנורה ביטוח מניות,44,"378,152.12"
4,"מנורה ביטוח מניות בחו""ל",101,"289,031.61"
5,"מנורה ביטוח אג""ח ממשלת ישראל",180,"306,301.40"
6,מנורה ביטוח מסלול הלכה,184,"102,763.92"
7,מנורה ביטוח בסיסי למקבלי קצבה,9534,"738,535.53"
8,"מנורה ביטוח אג""ח עד 25% מניות למקבלי קצבה",9535,"1,157,826.22"
9,מנורה ביטוח לבני 50 ומטה,9536,"1,543,147.49"


* total 17 funds in website, 2 of which mergerd into others --> 15.
<br><br>
<u>missing reports:</u>
1. 18011
1. 18012

download from here: https://www.menoramivt.co.il/rep-list/about-insurance/participating-portfolio-info/insurer-assets

In [ ]:
# adding sums of 2 migrated funds (data for these 2 is 1 quarter behind)
99576.34 + 531016.24 + all_holdings_final[all_holdings_final["ParentCorpName"] == 'מנורה מבטחים ביטוח בע"מ']["שווי"].sum()

Conclusion: there seems to be missing reports for Menora. Check again next quarter.

### הפניקס ביטוח - דוחות חסרים

In [127]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520023185'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

,report_id,ShortName,ProductNum,שווי
0,2154066,"הפניקס ביטוח מניות חו""ל",12882,"93,198.05"
1,2154068,"הפניקס ביטוח אג""ח בינ""ל",12881,"592,631.93"
2,2154067,הפניקס - מסלול פאסיבי כללי\n,12092,"397,250.12"
3,2154070,הפניקס - מסלול פאסיבי לבני 60 ומעלה\n,12091,"619,501.07"
4,2154069,הפניקס - מסלול פאסיבי לבני 50 עד 60\n,12090,"1,136,081.56"
5,2154071,הפניקס - מסלול פאסיבי לבני 50 ומטה,12089,"4,274,038.23"
6,2154061,הפניקס-אקסלנס אינווסט שיקלי טווח קצר,9603,"4,642.83"
7,2154059,הפניקס-אקסלנס אינווסט מסלול הלכה,9602,"2,308.84"
8,2154072,הפניקס-אקסלנס אינווסט מסלול מניות,9601,"31,625.95"
9,2154063,הפניקס-אקסלנס אינווסט מסלול כללי,9600,"249,218.84"


* website: 30 funds
* added 2 missing reports: 7012, 7011 (קרן ט, קרן י)
<br>
https://www.fnx.co.il/investors-relations-hebrew/investment-information-reports/reshimat-nichsey-hamevateah

In [129]:
url = "https://cdn.menoramivt.co.il/public/docs/20211205/520042540_b18012_p321.xlsx"
filename = "520042540_b18012_p321.xlsx"
ur.urlretrieve(url, filename)

URLError: <urlopen error [Errno 60] Operation timed out>

### הראל ביטוח - דוחות חסרים

In [113]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520004078'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

,report_id,ShortName,ProductNum,שווי
0,2152323,הראל מסלול מחקה מדד תל אביב 35,13437,"25,604.19"
1,2152329,הראל מסלול מחקה מדד תל בונד 60,13436,"26,241.27"
2,2152324,הראל מסלול מחקה מדד ממשלתי צמוד 5-10,13435,"24,556.88"
3,2152322,הראל מסלול מחקה מדד S&P 500,13434,"126,417.49"
4,2152321,הראל מסלול מחקה מדד ממשלתי שקלי ריבית קבועה,13433,"12,385.38"
5,2152328,"הראל מסלול הוני ללא אג""ח",12981,"484,321.34"
6,2152319,"הראל מסלול אג""ח עד 20% במניות למקבלי קצבה",11404,"366,509.58"
7,2152318,הראל מסלול בסיסי למקבלי קצבה,11403,"954,976.74"
8,2152315,הראל מסלול לגילאי 60 ומעלה,11402,"241,868.35"
9,2152317,הראל מסלול לגילאי 50 עד 60,11401,"132,944.57"


* website: 34 funds.<br><br>
<B> Manually added missing reports: 259010 259011 259012 259013 </B>
from website: https://www.harel-group.co.il/about/harel-group/harel/mesthtefet/Pages/list-of-assets.aspx

### כלל ביטוח - דוחות חסרים

In [114]:
pd.DataFrame(
    all_holdings_final[all_holdings_final["ParentCorpLegalId"] == '520024647'].groupby(["report_id", "ShortName", "ProductNum"])['שווי'].agg('sum').map('{:,.2f}'.format)
).sort_values("ProductNum", ascending=False).reset_index()

,report_id,ShortName,ProductNum,שווי
0,2153455,כלל ביטוח מחקה מדד S&P500,13346,"60,196.59"
1,2153443,כלל ביטוח שיקלי טווח קצר,13244,"24,470.12"
2,2153450,"כלל ביטוח למקבלי קצבה אג""ח עד 20% מניות - משתת...",12472,0.47
3,2153441,"כלל ביטוח פנסיונרים מקבלי קצבה אג""ח עד 20% מני...",12471,0.82
4,2153442,כלל מסלול בסיסי למקבלי קצבה פוליסות לפני 2004,11350,"2,197,872.01"
5,2153440,כלל מסלול לבני 60 ומעלה פוליסות לפני 2004,11349,"693,027.48"
6,2153444,כלל לבני 50-60 פוליסות לפני 2004,11348,"386,921.93"
7,2153438,כלל לבני 50 ומטה פוליסות לפני 2004,11341,"132,399.24"
8,2153454,כלל מסלול בסיסי למקבלי קצבה,9767,"1,326,029.64"
9,2153457,כלל - 60 ומעלה,9661,"1,425,108.89"


* website: 21 funds
* missing reports: 14011, 14012
<br><br>
downloaded from their site: https://www.clalbit.co.il/aboutclalinsurance/generalreports/

## הלמן - פנסיה וגמל
?גמל - הבדל בסכום, האם כתוצאה מדוחות חסרים
<br>
₪16,136,063 בדוח מסכם
16,132,489.84 לפי סכום דוחות מסלול

הבדלים קטנים, לא מצאתי הסבר.
אולי הוציאו דוחות בשעות שונות של היום?

In [ ]:
16136063 - 16132489.84

In [ ]:
helman = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("הלמן")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
helman_by_report = pd.DataFrame(helman.groupby(["ProductNum", "ShortName"])["שווי"].agg('sum').map('{:,.2f}'.format)).reset_index()
helman_by_report

## מגדל פנסיה

In [91]:
migdal_p = all_holdings_final[(all_holdings_final["ParentCorpName"].str.startswith("מגדל")) &
                             (all_holdings_final["SystemName"] == "פנסיה")
                            ]
migdal_p_report = pd.DataFrame(migdal_p.groupby(["report_id", "ProductNum", "ShortName"])["שווי"].agg('sum')).reset_index()
migdal_p_report.merge(totals,
                     on="report_id",
                     how="left")

# migdal_p_report["שווי"].sum()

,report_id,ProductNum,ShortName,שווי,asset,sum,pct,pct_num,sum_num
0,2148419,12147,מקפת אישית מניות למקבלי קצבה,"17,477.05","סה""כ סכום נכסי המסלול או הקרן","17,477.05",1,1,"17,477.05"
1,2148420,8803,מגדל מקפת אישית לבני 60 ומעלה,"1,598,428.65","סה""כ סכום נכסי המסלול או הקרן","1,598,428.65",1,1,"1,598,428.65"
2,2148421,12145,מקפת אישית למקבלי קצבה,"4,079,662.67","סה""כ סכום נכסי המסלול או הקרן","4,079,662.67",1,1,"4,079,662.67"
3,2148422,8602,מקפת אישית לזכאים קיימים לקצבה,"78,442.11","סה""כ סכום נכסי המסלול או הקרן","78,442.11",1,1,"78,442.11"
4,2148423,13572,מגדל מקפת אישית מחקה מדד S&P500,"196,082.18","סה""כ סכום נכסי המסלול או הקרן","196,082.18",1,1,"196,082.18"
5,2148424,8802,מגדל מקפת אישית לבני 50 עד 60,"2,893,633.69","סה""כ סכום נכסי המסלול או הקרן","2,893,633.69",1,1,"2,893,633.69"
6,2148425,2142,מגדל מקפת אישית מניות,"2,234,914.47","סה""כ סכום נכסי המסלול או הקרן","2,234,914.47",1,1,"2,234,914.47"
7,2148426,2207,מקפת אישית למקבלי קצבה קיימים,"3,596,134.08","סה""כ סכום נכסי המסלול או הקרן","3,596,134.08",1,1,"3,596,134.08"
8,2148427,2102,מקפת אישית כללי,"63,232,444.53","סה""כ סכום נכסי המסלול או הקרן","63,232,444.53",1.00,0.9999999999999999,"63,232,444.53"
9,2148428,9455,מקפת משלימה לבני 60 ומעלה,"66,532.19","סה""כ סכום נכסי המסלול או הקרן","66,532.19",1,1,"66,532.19"


* All pension funds from website are on file.<br>
<B> last Q there was also a discrepancy (2.1B, now 2.15B) </B>